In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movies = pd.read_csv("/content/drive/MyDrive/recomendacao/movies.csv")

In [ ]:
tags = pd.read_csv("/content/drive/MyDrive/recomendacao/tags.csv")

In [ ]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [ ]:
tags["tag"] = tags["tag"] + " "

In [ ]:
tags.groupby('movieId', as_index=False)['tag'].sum()

,movieId,tag
0,1,pixar pixar fun
1,2,fantasy magic board game Robin Williams game
2,3,moldy old
3,5,pregnancy remake
4,7,remake
...,...,...
1567,183611,Comedy funny Rachel McAdams
1568,184471,adventure Alicia Vikander video game adaptation
1569,187593,Josh Brolin Ryan Reynolds sarcasm
1570,187595,Emilia Clarke star wars


In [ ]:
movies.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [ ]:
movies['year'] = movies.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removendo os parenteses
movies['year'] = movies.year.str.extract('(\d\d\d\d)',expand=False)
#Removendo o ano da coluna título
movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')
#Assegurando que removemos todos os espaços em branco da palavra
movies['title'] = movies['title'].apply(lambda x: x.strip())

<ipython-input-6-1c25c204f0e0>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')


In [ ]:
movies.shape

(9742, 4)

In [ ]:
user_ratings = pd.read_csv("/content/drive/MyDrive/recomendacao/ratings.csv")

In [ ]:
user_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
def clean_genre(genres):
  return genres.replace("|", " ")

In [ ]:
movies["clean_genre"] = movies["genres"].apply(clean_genre)

In [ ]:
movies["target"] = movies["title"] + " " + movies["clean_genre"]

In [ ]:
movies.head()

,movieId,title,genres,year,clean_genre,target
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,Adventure Animation Children Comedy Fantasy,Toy Story Adventure Animation Children Comedy ...
1,2,Jumanji,Adventure|Children|Fantasy,1995,Adventure Children Fantasy,Jumanji Adventure Children Fantasy
2,3,Grumpier Old Men,Comedy|Romance,1995,Comedy Romance,Grumpier Old Men Comedy Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,Comedy Drama Romance,Waiting to Exhale Comedy Drama Romance
4,5,Father of the Bride Part II,Comedy,1995,Comedy,Father of the Bride Part II Comedy


In [ ]:
def Convert(string):
    li = list(string.split(" "))
    return li

# TF-IDF


In [ ]:
d = {'Filmes': ["Dragon Ball", "Kung Fu Panda", "Ace Ventura"], 'Gênero':  ["Action|Animation", "Action|Animation|Comedy", "Comedy"]}
df = pd.DataFrame(data=d)

In [ ]:
df.head()

In [ ]:
df["clean_genre"] = df["Gênero"].apply(clean_genre)

In [ ]:
df["target"] = df["Filmes"] + " " + df["clean_genre"]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 2)
vectorized_data = vectorizer.fit_transform(df['target']) 

In [ ]:
print(vectorizer.get_feature_names())

In [ ]:
# Create Dataframe com TF-IDFarray
tfidf_df = pd.DataFrame(vectorized_data.toarray(), columns=vectorizer.get_feature_names())

In [ ]:
# Assign the movie titles to the index and inspect
tfidf_df.index = df['Filmes']

In [ ]:
tfidf_df.head()

# Aplicando pra Base

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
movies.head()

,movieId,title,genres,year,clean_genre,target
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,Adventure Animation Children Comedy Fantasy,Toy Story Adventure Animation Children Comedy ...
1,2,Jumanji,Adventure|Children|Fantasy,1995,Adventure Children Fantasy,Jumanji Adventure Children Fantasy
2,3,Grumpier Old Men,Comedy|Romance,1995,Comedy Romance,Grumpier Old Men Comedy Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,Comedy Drama Romance,Waiting to Exhale Comedy Drama Romance
4,5,Father of the Bride Part II,Comedy,1995,Comedy,Father of the Bride Part II Comedy


In [ ]:
#min_df, max_df=0.7
vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1,4))


vectorized_data = vectorizer.fit_transform(movies['target']) 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

title = "Jumanji Adventure Children Fantasy"
query_vec = vectorizer.transform([title])
similarity = cosine_similarity(query_vec, vectorized_data).flatten()
indices = np.argpartition(similarity, -5)[-5:]
results = movies.iloc[indices]

In [ ]:
results[["title", "clean_genre", "year"]]

,title,clean_genre,year
6065,Zathura,Action Adventure Children Fantasy,2005
8641,Seventh Son,Adventure Children Fantasy,2014
8800,Pan,Adventure Children Fantasy,2015
1,Jumanji,Adventure Children Fantasy,1995
6389,Bridge to Terabithia,Adventure Children Fantasy,2007


## Recomendando

In [ ]:
print(vectorizer.get_feature_names())

['000', '000 leagues', '000 leagues under', '000 leagues under the', '10', '10 to', '10 to yuma', '10 to yuma action', '100', '101', '101 dalmatians', '10th', '11', '11 documentary', '12', '12 angry', '12 angry men', '12 chairs', '12 chairs adventure', '12 chairs adventure comedy', '13', '13 action', '13 action animation', '13 action crime', '13th', '13th horror', '13th part', '17', '17 drama', '18', '19', '1900', '20', '20 000', '20 000 leagues', '20 000 leagues under', '2000', '2000 action', '2000 animation', '2000 animation children', '2001', '2012', '21', '22', '25th', '28', '28 days', '2nd', '30', '30 comedy', '300', '3000', '34th', '34th street', '39', '3d', '3d action', '3d horror', '3d horror mystery', '3d horror mystery thriller', '40', '42', '42nd', '42nd street', '42nd street drama', '47', '47 action', '48', '48 hrs', '48 hrs action', '48 hrs action comedy', '49', '50', '51', '60', '60 seconds', '60 seconds action', '60 seconds action crime', '61', '6th', '80', '80 days', '8

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# Cria um Dataframe com TF-IDFarray
tfidf_df = pd.DataFrame(vectorized_data.toarray(), columns= vectorizer.get_feature_names())

In [ ]:
tfidf_df.index = movies['title']

In [ ]:
tfidf_df.head()

,000,000 leagues,000 leagues under,000 leagues under the,10,10 to,10 to yuma,10 to yuma action,100,101,...,zoo,zoolander,zoolander comedy,zoom,zorro,zorro the,zorro the action,zulu,école,été
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jumanji,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grumpier Old Men,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Waiting to Exhale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Father of the Bride Part II,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Importa cosine_similarity 
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Crie a matriz de valores de similaridade de cosseno
cosine_similarity_array = cosine_similarity(tfidf_df)

In [ ]:
# Coloca em forma de dataframe
cosine_similarity_df = pd.DataFrame(cosine_similarity_array, index=tfidf_df.index, columns=tfidf_df.index)

In [ ]:
cosine_similarity_df.head()

title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,Gintama: The Movie,anohana: The Flower We Saw That Day - The Movie,Silver Spoon,Love Live! The School Idol Movie,Jon Stewart Has Left the Building,Black Butler: Book of the Atlantic,No Game No Life: Zero,Flint,Bungo Stray Dogs: Dead Apple,Andrew Dice Clay: Dice Rules
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.081556,0.007094,0.010532,0.004672,0.0,0.007886,0.045738,0.0,0.029270,...,0.019285,0.020061,0.014115,0.027513,0.000000,0.074571,0.091986,0.000000,0.028184,0.007444
Jumanji,0.081556,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.196584,0.0,0.057808,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.034714,0.042821,0.000000,0.000000,0.000000
Grumpier Old Men,0.007094,0.000000,1.000000,0.050824,0.007354,0.0,0.093386,0.000000,0.0,0.000000,...,0.006430,0.000000,0.022222,0.000000,0.000000,0.008493,0.010477,0.000000,0.000000,0.011719
Waiting to Exhale,0.010532,0.000000,0.050824,1.000000,0.010919,0.0,0.056500,0.000000,0.0,0.000000,...,0.009546,0.010743,0.159036,0.000000,0.000000,0.012610,0.015555,0.029093,0.000000,0.017400
Father of the Bride Part II,0.004672,0.000000,0.007354,0.010919,1.000000,0.0,0.008176,0.000000,0.0,0.000000,...,0.009582,0.014116,0.014634,0.009680,0.009559,0.056886,0.006899,0.000000,0.000000,0.007718


In [ ]:
# Find the values for the movie Rio
cosine_similarity_series = cosine_similarity_df.loc['Toy Story']

In [ ]:
# Sort these values highest to lowest
ordered_similarities = cosine_similarity_series.sort_values(ascending=False)

In [ ]:
# Print the results
print(ordered_similarities[1:10])

title
Toy Story 2       1.000000
Toy Story 3       0.893399
Moana             0.591200
Antz              0.591200
Turbo             0.526338
Valiant           0.517031
Presto            0.493593
Monsters, Inc.    0.488939
Minions           0.467953
Name: Toy Story, dtype: float64


In [ ]:
# Cria um subset de filmes que o cliente assistiu
filmes_vistos = ['Toy Story',	'Jumanji',	'Grumpy Old Men']
movies_enjoyed_df = tfidf_df.loc[filmes_vistos]


In [ ]:
movies_enjoyed_df

,000,000 leagues,000 leagues under,000 leagues under the,10,10 to,10 to yuma,10 to yuma action,100,101,...,zoo,zoolander,zoolander comedy,zoom,zorro,zorro the,zorro the action,zulu,école,été
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jumanji,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grumpier Old Men,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Gera o quanto ele gostou em média de cada entrada do vetor
user_prof = movies_enjoyed_df.mean()

In [ ]:
user_prof.values.reshape(1,-1)

array([[0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
filmes_nao_assistidos = tfidf_df.drop(filmes_vistos,axis=0)

In [ ]:
similaridades_do_user = cosine_similarity(user_prof.values.reshape(1,-1), filmes_nao_assistidos)

In [ ]:
similaridades_do_user

array([[0.54412626, 0.01556772, 0.00690515, ..., 0.        , 0.01581   ,
        0.01100351]])

In [ ]:
similaridades_user_df = pd.DataFrame(similaridades_do_user.T, index=filmes_nao_assistidos.index, columns=["Score de Similaridade"])

In [ ]:
similaridades_user_df.sort_values(by="Score de Similaridade", ascending=False).head(10)

,Score de Similaridade
title,
Toy Story 2,0.610845
Toy Story 3,0.545728
Grumpier Old Men,0.544126
Moana,0.416016
Antz,0.416016
Old Men: Robbers,0.412139
Pan,0.407326
Bridge to Terabithia,0.388163
Turbo,0.370374


## Detalhe interessante

In [ ]:
def search(title):
  query_vec = vectorizer.transform([title])
  similarity = cosine_similarity(query_vec,vectorized_data).flatten()
  indices = np.argpartition(similarity, -5)[-5:]
  results = movies.iloc[indices]
  return results

In [ ]:
import ipywidgets as widgets 
from IPython.display import display

movie_input = widgets.Text(
    value = "Coloque o filme aqui",
    description = "Título do FIlme:",
    disabled=False
)

movie_list = widgets.Output()

def on_type(data):
  with movie_list:
    movie_list.clear_output()
    title = data["new"]
    if len(title) > 5:
      display(search(title))

movie_input.observe(on_type, names='value')

display(movie_input, movie_list)

Text(value='Coloque o filme aqui', description='Título do FIlme:')

Output()